In [1]:
import math
import torch
from diffusion_edf import dist
from diffusion_edf import transforms
import plotly.graph_objs as go

In [2]:
time = 0.5
lin_mult = 1.0

eps = time / 2
std = math.sqrt(time) * lin_mult

N = 100000
T = dist.sample_isotropic_se3_gaussian(eps=eps, std=std, N=N)
q = T[..., :4]
x = T[..., 4:]
z = torch.tensor([0., 0., 1.]).expand(N,3)

z_transformed = transforms.quaternion_apply(quaternion=q, point=z) + x

In [3]:
if N < 30000:
    z_vis = z_transformed

    layout = go.Layout(
        width=600,
        height=600,
        scene=dict(camera=dict(eye=dict(x=1.25, y=1.25, z=1.25)), #the default values are 1.25, 1.25, 1.25
                aspectmode='manual', #this string can be 'data', 'cube', 'auto', 'manual'
                #a custom aspectratio is defined as follows:
                aspectratio=dict(x=1, y=1, z=1),
                xaxis = dict(title='x', range = [-2,2]),
                yaxis = dict(title='y', range = [-2,2]),
                zaxis = dict(title='z', range = [-2,2]),
                )
    )
    fig = go.Figure(data=go.Scatter3d(x=z_vis[...,0], y=z_vis[...,1], z=z_vis[...,2], mode='markers', marker=dict(size=3)), layout=layout)
    fig.show()

In [4]:
n_iter = 100
z_diffuse = z
for n in range(n_iter):
    #t = time * (n+1) / n_iter
    dt = time / n_iter
    dw = torch.randn(N,6) * torch.tensor([lin_mult, lin_mult, lin_mult, 1., 1., 1.]) * math.sqrt(dt)
    T_diffuse = transforms.se3_exp_map(log_transform=dw)
    q_diffuse = transforms.matrix_to_quaternion(T_diffuse[..., :3, :3])
    x_diffuse = T_diffuse[..., :3, 3]
    z_diffuse = transforms.quaternion_apply(quaternion=q_diffuse, point=z_diffuse) + x_diffuse # T should be applied from t_max to t=0, but since the P(T_diffuse) is time invariant, any action ordering is fine.


In [5]:
print(z_transformed.std(dim=0))
print(z_transformed.mean(dim=0))

tensor([0.8673, 0.8713, 0.7832], dtype=torch.float64)
tensor([-4.7668e-04,  1.5403e-03,  6.0781e-01], dtype=torch.float64)


In [6]:
print(z_diffuse.std(dim=0))
print(z_diffuse.mean(dim=0))

tensor([0.8671, 0.8690, 0.7833])
tensor([-1.7509e-03,  2.1567e-04,  6.0799e-01])


In [7]:
if N < 30000:
    z_vis = z_diffuse

    layout = go.Layout(
        width=600,
        height=600,
        scene=dict(camera=dict(eye=dict(x=1.25, y=1.25, z=1.25)), #the default values are 1.25, 1.25, 1.25
                aspectmode='manual', #this string can be 'data', 'cube', 'auto', 'manual'
                #a custom aspectratio is defined as follows:
                aspectratio=dict(x=1, y=1, z=1),
                xaxis = dict(title='x', range = [-2,2]),
                yaxis = dict(title='y', range = [-2,2]),
                zaxis = dict(title='z', range = [-2,2]),
                )
    )
    fig = go.Figure(data=go.Scatter3d(x=z_vis[...,0], y=z_vis[...,1], z=z_vis[...,2], mode='markers', marker=dict(size=3)), layout=layout)
    fig.show()